# Grundlegende ideen bei einem Neural Network


Ein Deep Neural Network (DNN) ist eine leistungsstarke Technik des maschinellen Lernens, die in verschiedenen Bereichen Anwendung findet. Die Bedeutung eines Deep Neural Networks liegt in seiner Fähigkeit, komplexe Muster in Daten zu erkennen und Vorhersagen oder Klassifikationen basierend auf diesen Mustern zu treffen. Hier sind einige der Schlüsselaspekte, die die Bedeutung von DNNs verdeutlichen:

1. Mustererkennung: DNNs sind in der Lage, Muster in großen und komplexen Datensätzen zu erkennen, die von Menschen schwer oder unmöglich zu identifizieren sind. Dies macht sie besonders wertvoll in Anwendungen wie Bilderkennung, Spracherkennung und Natural Language Processing (NLP).

2. Klassifikation und Vorhersage: DNNs ermöglichen die Klassifikation von Daten in verschiedene Kategorien oder die Vorhersage zukünftiger Ereignisse. Dies wird in Anwendungen wie Spam-Erkennung, Gesichtserkennung, medizinischer Diagnose und Finanzprognosen genutzt.

3. Skalierbarkeit: DNNs können große Mengen an Daten verarbeiten und sind in der Lage, Modelle mit Tausenden oder Millionen von Parametern zu trainieren. Dies ermöglicht die Bewältigung komplexer Aufgaben und die Verarbeitung großer Datensätze.

4. Automatisierung: DNNs können automatisch und kontinuierlich lernen, was bedeutet, dass sie sich an neue Daten und Veränderungen in der Umgebung anpassen können, ohne dass eine manuelle Anpassung erforderlich ist.

5. Vielseitigkeit: DNNs finden in verschiedenen Bereichen Anwendung, darunter Bildverarbeitung, Sprachverarbeitung, autonome Fahrzeuge, medizinische Diagnostik, Empfehlungssysteme und vieles mehr. Sie sind eine Kernkomponente von Künstlicher Intelligenz (KI) und Deep Learning.

6. Forschung und Innovation: DNNs haben zu bedeutenden Fortschritten in der KI-Forschung geführt und neue Möglichkeiten für innovative Anwendungen eröffnet. Sie sind ein Motor für die Entwicklung neuer Technologien und Lösungen.

Die Bedeutung eines Deep Neural Networks liegt in seiner Fähigkeit, komplexe Aufgaben zu bewältigen und menschenähnliche Leistungen bei der Verarbeitung von Informationen zu erzielen. Dies hat Auswirkungen auf viele Branchen und trägt zur Transformation von Geschäftsmodellen und zur Lösung komplexer Herausforderungen bei.

--

Blackbox: Als Blockbox wir in Data Science die Hiddenlayers gemeint die der Programmier nicht sehen. Da nicht nachvollziebar ist wie das Model den Output generiert. Das Auch Deepl Learning Neurale Netzwerke Hidden Layers nutzen und die für den Programmierer auch eine Blackbox ist (siehe bild)

![Hidden Layer](/Users/riccardo/Desktop/Repositorys_Github/Training/Scripts/Models/pictures_expl/hiddenlayer.png)


### **Arten von rekurrenten neuronalen Netzwerken (RNNs)**

Eine Eingabesequenz ist einer Ausgabe zugeordnet. Sie können sie jedoch flexibel an verschiedene Konfigurationen für bestimmte Zwecke anpassen. Im Folgenden sind einige gänige RNN-Typen aufgeführt.

#### Eins-zu-viele
- Dieser RNN-Typ kanalisiert einen Eingang an mehrere Ausgänge. Es ermöglicht sprachliche Anwendungen wie Bildunterschirften, indem ein Satz aus einem einzigen Schüsselwort generiert wird.

#### Viel-zu-viele
- Das Modell verwendet mehrere Eingaben, um mehrere Ausgaben vorherzusagen. Sie können beispielsweise einen Sprachübersetzter mit einem RNN erstellen, der einen Satz analysiert und die Wörter in einer anderen Sprache korrekt strukturiert.

#### Viel-zu-eins
- Mehrere Eingänge werden einem Ausgang zugeordet. Dies ist hilfreich bei Anwendungen wie der Stimmungsanalyse, bei der das Modell positive, negative und neutrale Stimmungen der Kunden anhand von Kundenreferenzen vorhersagt.


### Rekurrentes neuronales Netzwerk im Vergleich zu neuronalem Feed-Forward-Netzwerk

- Wie Rnns sind neuronale Feed-Forward-Netze künstliche neuronale Netze, die Informationen von einem Ende zum Ende der Architektur weiterleiten. 
Ein neuuronales Feed-Forward-Netzwerk kann einfache Klassifizierungens-, Regressions- oder Erkennungsaufgaben ausführen, aber es kann sich nicht an die vorherige Eingabe erinnern, die es verarbeitet hat.

- Zumbeispiel vergisst es "Apfel", wenn sein Neuron das Wort *ist* verarbeitet.
Das RNN überwindet diese Speicherbeschränkung, indem es einen verborgenen Gedächtniszustand in das Neuron einbezieht.

--

In einem **einfachen neuronalen Netzwerk** ist die Berechnung ähnlich wie bei einer **linearen Regression**, bei der eine Ausgabe 
**y** anhand einer linearen Kombination der Eingaben **x**, sowie eines **Intercepts (weight)** und eines **slope (Bias)** berechnet wird. Der Hauptunterschied besteht darin, dass ein neuronales Netzwerk mehrere Schichten von Neuronen hat, die nichtlinear miteinander verbunden sind, wodurch es komplexere Beziehungen zwischen Eingabe und Ausgabe modellieren kann. 


<img src="/Users/riccardo/Desktop/Repositorys_Github/Training/Scripts/Models/pictures_expl/Terminologie_weights.png" width="400" />

<img src="/Users/riccardo/Desktop/Repositorys_Github/Training/Scripts/Models/pictures_expl/Terminologie_baises.png" width="400" />


--

Es gibt viele aktivierungsfunktionen für ein Neurales Network
- ReLU (Rectified Linear Unit)
- Softplus
- Sigmoid 

![Aktiverierungsfunktion](/Users/riccardo/Desktop/Repositorys_Github/Training/Scripts/Models/pictures_expl/aktivierungsfunktionen.jpeg)

Recurrent Neural Networks (RNN) sind Neuornale Netzwerke. Die für die Verarbeitung squenzieller Daten entwickelt wuruden.

Zwei wesentliche und wichtige Unterscheidungen 
- Feed-forward neural networks 

    - Neuronale Netze mit Feed-forward neural networks sind der häufigste Typ von neuronalen Netzen. Sie bestehen aus Schichten von Knoten, und die Informationen fließen in einer einzigen Richtung, von der Eingabeschicht (input) zur Ausgabeschicht (output).

    - Jeder Knoten in einem neuronalen Feed-Forward-Netzwerk führt eine einfache Berechnung durch, und die Ausgabe jedes Knotens wird an den nächsten Knoten in der Schicht weitergeleitet.
    
    - Die Architektur eines einfachen RNN besteht aus einer Schleife, die es dem Netzwerk ermöglicht, Informationen aus vorherigen Schritten zu berücksichtigen. Dieser Rückkopplungsmechanismus erlaubt es dem Netzwerk, auf vergangene Zustände zu reagieren und somit eine gewisse Form von Gedächtnis zu haben. Ein Problem bei einfachen RNNs ist jedoch das Verschwinden oder Explodieren des Gradienten bei der Rückwärtspropagierung, was zu Schwierigkeiten beim Training führen kann.

    - Was nutzen wir Feed-forward neural networks:
    
        -> Image Classification

        -> Natural language processing 
        
        -> speech recognition
    - Nutzt man für Vorhersagen von continuous values (Bsp Aktienpreise vorhersagen)




Formel :


![Beschreibung des Bildes](/Users/riccardo/Desktop/Repositorys_Github/Training/Scripts/Models/pictures_expl/eq_29.png)



- y(t) = Steht für die Ausgabe des neuronalen Netzes zur Zeit (t). 
Es repräsentiert das, was das Netzwerk zu einem bestimmten Zeitpunkt vorhersagt oder generiert.

- Die Funktion **ReLU** ist eine Aktivierungs-funktion, die in neuronalen Netzten verwendet wird. Sie steht für **(Rectified Linear Unit)**. 
Diese Funktion gibt einfach den Eingabewert zurück, **wenn er positiv ist, und gibt 0 zurück, wenn er negativ ist**. Es ist eine weit verbreitete Aktivierungsfunktion aufgrund ihrer Einfachheit und ihrer guten Leistung in vielen Anwendungen.

- **Wx** und **Wy** sind Gewichtsmatrizen, die die Verbindungen zwischen den Eigaben **Xt** und den vorherigen Ausgaben **yt−1** und den versteckten Schichten des neuronalen Netzes steuern. WXT und WTY bezeichne die Transponierten dieser Gewichtsmatrizen.

- **Xt** ist die Eingabe des neuronales Netzes zum Zeitpunkt **t**. Sie repräsentiert die Daten oder Informationen, die dem Netzwerk zu diesem Zeitpunkt präsentiert werden.

- **yt-1** ist die Ausgabe des neuronalen Netzes zum vorherigen Zeitpunkt t - 1. Sie wird verwendet, um die Informationen aus vergangenen Zustände zu berücksichtigen.

- **b** ist der Bias-Vektor für die versteckten Schichten des neuronalen Netzes, ER ermöglicht es dem Netzwerk, Verschiebungen in den Daten zu berücksichtigen.

Insgesamt wird die Ausgabe des neuronalen Netzes zum Zeitpunkt **t**
als gewichtete Summe der Eingaben **Xt** und der vorherigen Ausgabe **yt-1** berechnet, die dann durch **die ReLU-Aktivierungsfunktion** gehen. Dieser Prozess wird verwendet, um Vorhersagen zu machen oder Daten zu generieren, basierend auf den Informationen zu diesem Zeitpunkt und vergangenen Zuständen des Netzwerks.

Es gibt viele aktivierungsfunktionen für ein Neurales Network
- ReLU (Rectified Linear Unit)
- Softplus
- Sigmoid 

![Aktiverierungsfunktion](/Users/riccardo/Desktop/Repositorys_Github/Training/Scripts/Models/pictures_expl/aktivierungsfunktionen.jpeg)

# Kapitel 15

## 1. Verarbeiten von Sequenzen mit Rnns und Cnns

- RNN = Rekurrente neuronale Netze
CNN = 

Folgendes wird im Kaptiel Thematisert
1. Grundlegende Konzepte auf den RNN aufbaut und werden erfahren wie man sie per Backpropagation durch die Zeit trainiert
2. Schwierigkeiten der Modelle Thematisieren
    - Instablie Gradienten die durch verschiedene Techniken abgemildert werden können - unter anderem Recurrent Dropout und Recurrent Layer Normalization
    - Ein sehr begrenztes Kurzeitgedächtnis das mithilfe von LSTM und GRU-Zellen erweiter werden kann

Rnn sind nicht die einzigen Neuronal Netz die sequenzielle Daten verarbeiten können

In [1]:
import numpy as np
import keras
import pandas as pd


In [2]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2,offsets1, offsets2 = np.random.rand(4,batch_size, 1)
    time = np.linspace(0,1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))    # Welle 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20))   # Welle 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)     # + Rauschen
    return series[..., np.newaxis].astype(np.float32)

In [5]:
n_steps = 50

series = generate_time_series(10000, n_steps + 1)
series.shape

(10000, 51, 1)

In [3]:
# Erstellungs eines Traininigs einen Validierungs und einen Testdatensattz
n_steps = 50

series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000: -1]


In [4]:
print("Wir haben also 10000 Reihen mit 50 Spalten", series.shape)

Wir haben also 10000 Reihen mit 50 Spalten (10000, 51, 1)


In [5]:
print("Der X Test Datensatz besthet aus: ", X_train.shape, "Der Y Test Datensatz besthet aus: ", y_train.shape)
print("Der X Validierungs Datensatz besthet aus: ", X_valid.shape,"Der Validierungs Datensatz besthet aus: ", y_valid.shape)
print("Der X_test Datensatz besthet aus: ", X_test.shape,"Der y_test Datensatz besteht  aus: ", y_test.shape)


Der X Test Datensatz besthet aus:  (7000, 50, 1) Der Y Test Datensatz besthet aus:  (7000, 1)
Der X Validierungs Datensatz besthet aus:  (2000, 50, 1) Der Validierungs Datensatz besthet aus:  (2000, 1)
Der X_test Datensatz besthet aus:  (1000, 50, 1) Der y_test Datensatz besteht  aus:  (999, 51, 1)


In [6]:
print("**Daten typ**", type(X_train), type(y_train), type(X_valid), type(y_valid), type(X_test), type(y_test))
# alle sind numpy arrays und haben die gleiche Anzahl an Dimensionen
print("Shape der Daten",X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape, y_test.shape)

**Daten typ** <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
Shape der Daten (7000, 50, 1) (7000, 1) (2000, 50, 1) (2000, 1) (1000, 50, 1) (999, 51, 1)


In [7]:
y_pred = X_valid[:,-1]
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

0.020439716

Der mittlere quadratische Fehler (MSE) ist eine gängige Metrik zur Bewertung der Leistung von Regressionsmodellen. Sie misst die durchschnittliche quadratische Abweichung zwischen den tatsächlichen und den vorhergesagten Werten.

Hier ist eine schrittweise Erklärung des MSE:
1. Differenz zwischen tatsächlichem und vorhergesagtem Wert: Für jede Instanz in den Daten wird die Differenz zwischen dem tatsächlichen Wert 
​und dem vorhergesagten Wert berechnet.

2. Quadrat der Differenz: Die Differenzen werden quadriert. Dies stellt sicher, dass negative und positive Abweichungen gleich behandelt werden und dass größere Abweichungen stärker ins Gewicht fallen.

3. Durchschnitt der quadrierten Differenzen: Die quadrierten Differenzen werden dann gemittelt, um den MSE zu erhalten. Dies geschieht, indem man die Summe der quadrierten Differenzen durch die Anzahl der Instanzen teilt.

Mathematisch ausgedrückt:


<img src="/Users/riccardo/Desktop/Repositorys_Github/Training/Scripts/Models/pictures_expl/mse.png" width="400" />



wo: 
n die anzahl 
yi der tatsächliche Wert den i-ten Instanz ist
y^i der vorherergesagte Wert der i-ten Instanz ist.


WICHTIG:
- Ein niedriger MSE-Wert deutet drauf hin, dass das Modell gute Vorhersagen macht, da die tatsächlichen und vorhergesagten Werte eng beieinander liegen. 

- Ein hoher MSE-Wert deutet hingegen darauf hin, dass das Model schlechte Vorhersagen macht, da die Abweichung zwischen den tatäschlich und den vorhergesagten Werten groß ist.


1. Flatten-Schicht: ist also wenn ich mir das Modul vorstellen das Input des Models
Vorschläge anzeigen

Die Flatten-Schicht kann als "Eingangstrichter" des Modells betrachtet werden. Sie nimmt die Eingabedaten entgegen und formatiert sie so, dass sie von den folgenden Schichten des Modells verarbeitet werden können.

Details:

Die Flatten-Schicht nimmt Daten mit einer beliebigen Form entgegen und "ebnet" sie in einen eindimensionalen Vektor.
Dies ist notwendig, da die meisten anderen Schichten des Modells mit Daten in Form von Vektoren arbeiten.
Die Größe des Vektors hängt von der Anzahl der Merkmale in den Eingabedaten ab.
Beispiel:

Angenommen, Sie haben Eingabedaten mit der Form (50, 10). Dies bedeutet, dass Sie 50 Datensätze mit jeweils 10 Merkmalen haben.

Die Flatten-Schicht würde diese Daten in einen Vektor mit 500 Elementen umwandeln.
Jedes Element des Vektors würde einem Merkmal aus einem der 50 Datensätze entsprechen.
Vorsichtsmaßnahmen:

Die Flatten-Schicht kann nur für Daten verwendet werden, die bereits in Form eines Arrays vorliegen.
Wenn Ihre Daten in einem anderen Format vorliegen, müssen Sie sie vor der Verwendung der Flatten-Schicht in ein Array konvertieren.

Erklärung des Code abschnitt:

- **keras.Models.Sequential** leitet ein model ein
- **keras.layer.Flatten** ist das Input, wenn man sich das Model vorsellt.
- **keras.layer.Dense** ist das Output des Models. 

    -- In unseren konrekten code ist noch keine Hidden layer eingebaut
Somit können wir sehen das wir nur ein Input sowie Output haben des RNN Models. Jedoch noch keine Hiddenlayer

In [8]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50,1], name = "Input"),
    keras.layers.Dense(1, name = "Output")
])


 In diesem Fall wird **mean_squared_error** verwendet, was den mittleren quadratischen Fehler berechnet. Diese Funktion eignet sich gut für Regressionsaufgaben, bei denen kontinuierliche Werte vorhergesagt werden sollen.

In [9]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [10]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20


167/219 [=====================>........] - ETA: 0s - loss: 0.4429 

2024-03-03 13:25:17.147979: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


219/219 [==============================] - 0s 315us/step - loss: 0.3724
Epoch 2/20
219/219 [==============================] - 0s 284us/step - loss: 0.0884
Epoch 3/20
219/219 [==============================] - 0s 289us/step - loss: 0.0454
Epoch 4/20
219/219 [==============================] - 0s 282us/step - loss: 0.0271
Epoch 5/20
219/219 [==============================] - 0s 479us/step - loss: 0.0186
Epoch 6/20
219/219 [==============================] - 0s 319us/step - loss: 0.0144
Epoch 7/20
219/219 [==============================] - 0s 274us/step - loss: 0.0119
Epoch 8/20
219/219 [==============================] - 0s 274us/step - loss: 0.0103
Epoch 9/20
219/219 [==============================] - 0s 275us/step - loss: 0.0091
Epoch 10/20
219/219 [==============================] - 0s 272us/step - loss: 0.0081
Epoch 11/20
219/219 [==============================] - 0s 270us/step - loss: 0.0073
Epoch 12/20
219/219 [==============================] - 0s 268us/step - loss: 0.0066
Epoch 13/20


In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (Flatten)             (None, 50)                0         
                                                                 
 Output (Dense)              (None, 1)                 51        
                                                                 
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________


Loss: ist ein Maß dafür wie gut oder schlecht Ihr Modell während des Trainings abschneidet. 

**model.compile(loss='mean_squared_error', optimizer='adam')** auch (MSE) genannt.
In meinem Fall habe ich als *loss funktion* den **'mean_squared_error'** eingesetzt.

- Höhe des Verlustes: Ein niedriger Verlustwert bedeutet, dass Ihr Modell gute Vorhersagen macht und die tatsächlichen Werte gut reproduziert. Ein hoher Verlustwert deutet darauf hin, dass Ihr Modell schlechte Vorhersagen macht und die tatsächlichen Werte nicht gut reproduziert.

- Veränderung des Verlustes über die Epochen: Wenn der Verlust im Laufe des Trainings abnimmt, deutet dies darauf hin, dass Ihr Modell besser wird und die Vorhersagen genauer werden. Wenn der Verlust jedoch stagniert oder steigt, kann dies darauf hinweisen, dass Ihr Modell nicht mehr lernt oder überangepasst ist.

In [ ]:
help(keras.Model.compile)

## Ein einfaches RNN implementieren
### SimpleRnn

In [12]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape = [None, 1])
])

In [13]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train,  epochs=20)

Epoch 1/20


219/219 [==============================] - 1s 3ms/step - loss: 0.0190 - accuracy: 0.0000e+00
Epoch 2/20
219/219 [==============================] - 1s 2ms/step - loss: 0.0128 - accuracy: 0.0000e+00
Epoch 3/20
219/219 [==============================] - 1s 2ms/step - loss: 0.0115 - accuracy: 0.0000e+00
Epoch 4/20
219/219 [==============================] - 1s 2ms/step - loss: 0.0113 - accuracy: 0.0000e+00
Epoch 5/20
219/219 [==============================] - 1s 2ms/step - loss: 0.0113 - accuracy: 0.0000e+00
Epoch 6/20
219/219 [==============================] - 1s 2ms/step - loss: 0.0113 - accuracy: 0.0000e+00
Epoch 7/20
219/219 [==============================] - 1s 2ms/step - loss: 0.0113 - accuracy: 0.0000e+00
Epoch 8/20
219/219 [==============================] - 1s 2ms/step - loss: 0.0113 - accuracy: 0.0000e+00
Epoch 9/20
219/219 [==============================] - 1s 2ms/step - loss: 0.0113 - accuracy: 0.0000e+00
Epoch 10/20
219/219 [==============================] - 1s 2ms/step - loss: 

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 1)                 3         
                                                                 
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


#### Deep Rnns

In [16]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape = [None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

In [17]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [18]:
model.fit(X_train, y_train,  epochs=20)

Epoch 1/20


219/219 [==============================] - 3s 8ms/step - loss: 0.0115
Epoch 2/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0039
Epoch 3/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0034
Epoch 4/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0032
Epoch 5/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0032
Epoch 6/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0031
Epoch 7/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0030
Epoch 8/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0030
Epoch 9/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0029
Epoch 10/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0029
Epoch 11/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0029
Epoch 12/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0029
Epoch 13/20
219/219 [===============

In [19]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape = [None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

In [20]:
model.compile(loss = 'mean_squared_error',optimizer='adam')

In [21]:
model.fit(X_train, y_train,  epochs=20)

Epoch 1/20


219/219 [==============================] - 2s 6ms/step - loss: 0.0084
Epoch 2/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0036
Epoch 3/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0034
Epoch 4/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0032
Epoch 5/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0031
Epoch 6/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0030
Epoch 7/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0030
Epoch 8/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0029
Epoch 9/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0029
Epoch 10/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0028
Epoch 11/20
219/219 [==============================] - 1s 7ms/step - loss: 0.0027
Epoch 12/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0027
Epoch 13/20
219/219 [===============

In [22]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_4 (SimpleRNN)    (None, None, 20)          440       
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 20)                820       
                                                                 
 dense (Dense)               (None, 1)                 21        
                                                                 
Total params: 1,281
Trainable params: 1,281
Non-trainable params: 0
_________________________________________________________________


In [23]:
series = generate_time_series(1, n_steps + 10)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:] # X_new ist die Zeitreihe bis 50 und Y_new ist die Zeitreihe ab 50
X = X_new
for step_ahead in range(10):
    y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    X = np.concatenate([X, y_pred_one], axis=1)

Y_pred = X[:, n_steps:]
# Berechnung des MSE


1/1 [==============================] - 0s 13ms/step


In [24]:
np.mean(keras.losses.mean_squared_error(Y_new, Y_pred))

0.022313457

### Seite 513

In [25]:
series = generate_time_series(10000, n_steps + 10)
X_train, Y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]  # 10 Schritte in die Zukunft
X_test, Y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

In [27]:
series.shape, X_train.shape, Y_train.shape, X_valid.shape, Y_valid.shape, X_test.shape, Y_test.shape

((10000, 60, 1),
 (7000, 50, 1),
 (7000, 10),
 (2000, 50, 1),
 (2000, 10),
 (1000, 50, 1),
 (1000, 10))

In [28]:
model = keras.models.Sequential([ 
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape = [None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])

In [29]:
Y_pred = model.predict(X_new)

1/1 [==============================] - 0s 108ms/step


In [30]:
model.compile(loss = 'mean_squared_error',optimizer='adam')

In [31]:
model.fit(X_train, y_train,  epochs=20)

Epoch 1/20
219/219 [==============================] - 2s 7ms/step - loss: 0.1501
Epoch 2/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1453
Epoch 3/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1449
Epoch 4/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1448
Epoch 5/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1448
Epoch 6/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1447
Epoch 7/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1446
Epoch 8/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1446
Epoch 9/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1447
Epoch 10/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1446
Epoch 11/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1446
Epoch 12/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1446
Epoch 13/20
219/219 [====

In [35]:
y_pred = X_valid[:,-1]
np.mean(keras.losses.mean_squared_error(Y_valid, Y_pred))

0.2813437